# nPE: A Configurable Processing Engine
#### Verification | Version 0.5.1 | Updated 2018.7.26
___

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import scala.math.pow

## Parallel Register File

### Single Register File

#### Definition

In [ ]:
class RFConfig(
        val addrWidth: Int, 
        val dataWidth: Int,
        val numInputs: Int,
        val bpSupport: Boolean)

class RFControl(c: RFConfig) extends Bundle {
    
    override def cloneType = (new RFControl(c)).asInstanceOf[this.type]
    
    val wEnable = Bool()
    val rEnable = Bool()
    val wAddr = UInt(c.addrWidth.W)
    val rAddrInt = UInt(c.addrWidth.W)
    val rAddrExt = UInt(c.addrWidth.W)
    val bpSel = if (c.bpSupport) Some(Bool()) else None
    val inSel = if (c.numInputs > 1) Some(Vec(c.numInputs, Bool())) else None
}

class RFInput(c: RFConfig) extends Bundle {
    
    override def cloneType = (new RFInput(c)).asInstanceOf[this.type] 
    
    val data = Vec(c.numInputs, SInt(c.dataWidth.W))
}

// TODO: Split this
class RFOutput(c: RFConfig) extends Bundle {
    
    override def cloneType = (new RFOutput(c)).asInstanceOf[this.type]
    
    val int = SInt(c.dataWidth.W)
    val ext = SInt(c.dataWidth.W)
}

class RF(c: RFConfig) extends Module {
    
    val io = IO(new Bundle {
        val control = Input(new RFControl(c))
        val in = Input(new RFInput(c))
        val out = Output(new RFOutput(c))
    })
    
    val zeros = Vec(Seq.fill(pow(2, c.addrWidth).toInt) { 0.S(c.dataWidth.W) })
    val registers = RegInit(zeros)
    
    val sel = io.control.inSel.getOrElse(Vec(true.B))
    
    when (io.control.wEnable) {
        registers(io.control.wAddr) := PriorityMux(sel, io.in.data)
    }
    
    
    when (io.control.rEnable) {
        // This bypass is disallowed by FIRRTL. It has
        // the potential to cause a combinational loop
//         when(io.control.bpSel.getOrElse(false.B)) {
//             io.out.int := PriorityMux(sel, io.in.data)
//             io.out.ext := PriorityMux(sel, io.in.data)
//         } .otherwise {
//             io.out.int := registers(io.control.rAddrInt)
//             io.out.ext := registers(io.control.rAddrExt)
//         }
        io.out.int := registers(io.control.rAddrInt)
        io.out.ext := registers(io.control.rAddrExt)
    } .otherwise {
        io.out.int := 0.S
        io.out.ext := 0.S
    }
}

#### Verification: Single Input

In [ ]:
val exampleRFConOne = new RFConfig(4, 8, 1, true)

Driver(() => new RF(exampleRFConOne)) {
    uut => new PeekPokeTester(uut) {
         
        poke(uut.io.control.bpSel.get, false)
        poke(uut.io.control.wEnable, true)   
        poke(uut.io.control.rEnable, true)

        poke(uut.io.control.wAddr, 1)
        poke(uut.io.in.data(0), 1)
        
        step(1)
        
        poke(uut.io.control.rAddrInt, 1)
        expect(uut.io.out.int, 1)
        
        poke(uut.io.control.rAddrExt, 1)
        expect(uut.io.out.ext, 1)
        
        poke(uut.io.control.wAddr, 2)
        poke(uut.io.in.data(0), 2)
        
        step(1)
        
        poke(uut.io.control.rAddrInt, 1)
        expect(uut.io.out.int, 1)
        
        poke(uut.io.control.rAddrExt, 2)
        expect(uut.io.out.ext, 2)
        
        poke(uut.io.control.wAddr, 3)
        poke(uut.io.in.data(0), 3)
        
        step(1)
        
        poke(uut.io.control.rAddrInt, 1)
        expect(uut.io.out.int, 1)
        
        poke(uut.io.control.rAddrExt, 2)
        expect(uut.io.out.ext, 2)
        
        poke(uut.io.control.rAddrInt, 3)
        expect(uut.io.out.data.int, 3)
        
        step(1)
        
        poke(uut.io.control.bpSel.get, true)
        poke(uut.io.in.data(0), 10)
        expect(uut.io.out.data.int, 10)
        expect(uut.io.out.data.ext, 10)
        
        poke(uut.io.control.bpSel.get, false)
        expect(uut.io.out.data.int, 3)
        expect(uut.io.out.data.ext, 2)
        
    }
}

#### Verification: Multiple Inputs

In [ ]:
val exampleRFConTwo = new RFConfig(4, 8, 2, true)

Driver(() => new RF(exampleRFConTwo)) {
    uut => new PeekPokeTester(uut) {
         
        poke(uut.io.control.bpSel.get, false)
        poke(uut.io.control.wEnable, true)   
        poke(uut.io.control.rEnable, true)

        // Write, input(0)
        poke(uut.io.control.wAddr, 1)
        poke(uut.io.control.inSel.get(0), 1)
        poke(uut.io.control.inSel.get(1), 0)
        poke(uut.io.in.data(0), 1)
        poke(uut.io.in.data(1), 10)
        
        step(1)
        
        // Read
        poke(uut.io.control.rAddrInt, 1)
        expect(uut.io.out.int, 1)
        poke(uut.io.control.rAddrExt, 1)
        expect(uut.io.out.ext, 1)
        
        // Write, input(1)
        poke(uut.io.control.wAddr, 2)
        poke(uut.io.control.inSel.get(0), 0)
        poke(uut.io.control.inSel.get(1), 1)
        poke(uut.io.in.data(0), 2)
        poke(uut.io.in.data(1), 20)
        
        step(1)
        
        // Read
        poke(uut.io.control.rAddrInt, 1)
        expect(uut.io.out.int, 1)
        poke(uut.io.control.rAddrExt, 2)
        expect(uut.io.out.ext, 20)
        
        // Write, input(0)
        poke(uut.io.control.wAddr, 3)
        poke(uut.io.control.inSel.get(0), 1)
        poke(uut.io.control.inSel.get(1), 0)
        poke(uut.io.in.data(0), 3)
        poke(uut.io.in.data(1), 30)
        
        step(1)
        
        // Read
        poke(uut.io.control.rAddrInt, 1)
        expect(uut.io.out.int, 1)
        poke(uut.io.control.rAddrExt, 2)
        expect(uut.io.out.ext, 20)
        poke(uut.io.control.rAddrInt, 3)
        expect(uut.io.out.int, 3)
        
        step(1)
        
        // Bypass
        poke(uut.io.control.bpSel.get, true)
        
        poke(uut.io.control.inSel.get(0), 1)
        poke(uut.io.control.inSel.get(1), 0)
        poke(uut.io.in.data(0), 5)
        poke(uut.io.in.data(1), 7)
        expect(uut.io.out.int, 5)
        expect(uut.io.out.ext, 5)
        
        poke(uut.io.control.inSel.get(0), 0)
        poke(uut.io.control.inSel.get(1), 1)
        poke(uut.io.in.data(0), 5)
        poke(uut.io.in.data(1), 7)
        expect(uut.io.out.int, 7)
        expect(uut.io.out.ext, 7)
        
        poke(uut.io.control.bpSel.get, false)
        expect(uut.io.out.int, 3)
        expect(uut.io.out.ext, 20)
        
    }
}

### Putting them Together

#### Definition

In [ ]:
class PRFConfig(
        val ports: Int,
        val dataWidth: Int,
        val addrWidth: Int,
        val numInputs: Int,
        val bpType: String) {
    
    require(List("None", "Soft", "Hard") contains bpType)
    
    val bpNone = (bpType == "None")
    val bpSoft = (bpType == "Soft")
    val bpHard = (bpType == "Hard")
    
    val rfConfig = new RFConfig(addrWidth, dataWidth, numInputs, bpSoft)
}

class PRFControl(c: PRFConfig) extends Bundle {
    
    override def cloneType = (new PRFControl(c)).asInstanceOf[this.type]
    
    val rf = Vec(c.ports, new RFControl(c.rfConfig))
}

class PRFInput(c: PRFConfig) extends Bundle {
    
    override def cloneType = (new PRFInput(c)).asInstanceOf[this.type]
    
    val rf = Vec(c.ports, new RFInput(c.rfConfig))
}

class PRFOutput(c: PRFConfig) extends Bundle {
    
    override def cloneType = (new PRFOutput(c)).asInstanceOf[this.type]
    
    val rf = Vec(c.ports, new RFOutput(c.rfConfig))
}

class PRF(c: PRFConfig) extends Module {
    
    val io = IO(new Bundle {
        val control = Input(new PRFControl(c))
        val in = Input(new PRFInput(c))
        val out = Output(new PRFOutput(c))
    })
    
    if (c.bpNone || c.bpSoft) {
        
        val rf = Seq.fill(c.ports){ Module(new RF(c.rfConfig)) }
        
        rf.zipWithIndex.map {
            case (x: RF, i: Int) => { 
                x.io.control <> io.control.rf(i)
                x.io.in <> io.in.rf(i)
                io.out.rf(i) <> x.io.out
            } 
        }
        
    } else if (c.bpHard) {
        for (i <- 0 until c.ports) {
            io.out.rf(i).int := io.in.rf(i).data
            io.out.rf(i).ext := io.in.rf(i).data
        }
    }
}

#### Verification

In [ ]:
val examplePRFCon = new PRFConfig(2, 8, 4, 2, "Soft")

Driver(() => new PRF(examplePRFCon)) {
    uut => new PeekPokeTester(uut) {
         
        poke(uut.io.control.rf(0).wEnable, true)  
        poke(uut.io.control.rf(1).wEnable, true) 
        poke(uut.io.control.rf(0).rEnable, true)
        poke(uut.io.control.rf(1).rEnable, true)
        poke(uut.io.control.rf(0).bpSel.get, false)
        poke(uut.io.control.rf(1).bpSel.get, false)
        poke(uut.io.control.rf(0).inSel.get(0), true)
        poke(uut.io.control.rf(1).inSel.get(0), true)
        poke(uut.io.control.rf(0).inSel.get(1), false)
        poke(uut.io.control.rf(1).inSel.get(1), false)

        poke(uut.io.control.rf(0).wAddr, 1)
        poke(uut.io.control.rf(1).wAddr, 1)
        poke(uut.io.in.rf(0).data(0), 1)
        poke(uut.io.in.rf(1).data(0), 1)
        
        step(1)
        
        // Read
        poke(uut.io.control.rf(0).rAddrInt, 1)
        poke(uut.io.control.rf(1).rAddrInt, 1)
        expect(uut.io.out.rf(0).int, 1)
        expect(uut.io.out.rf(1).int, 1)
        
        poke(uut.io.control.rf(0).rAddrExt, 1)
        poke(uut.io.control.rf(1).rAddrExt, 1)
        expect(uut.io.out.rf(0).ext, 1)
        expect(uut.io.out.rf(1).ext, 1)
        
        // Write
        poke(uut.io.control.rf(0).wAddr, 2)
        poke(uut.io.control.rf(1).wAddr, 2)
        poke(uut.io.in.rf(0).data(0), 2)
        poke(uut.io.in.rf(1).data(0), 2)
        
        step(1)
        
        // Read
        poke(uut.io.control.rf(0).rAddrInt, 1)
        poke(uut.io.control.rf(1).rAddrInt, 1)
        expect(uut.io.out.rf(0).int, 1)
        expect(uut.io.out.rf(1).int, 1)
        
        poke(uut.io.control.rf(0).rAddrExt, 2)
        poke(uut.io.control.rf(1).rAddrExt, 2)
        expect(uut.io.out.rf(0).ext, 2)
        expect(uut.io.out.rf(1).ext, 2)
        
        // Write
        poke(uut.io.control.rf(0).wAddr, 3)
        poke(uut.io.control.rf(1).wAddr, 3)
        poke(uut.io.in.rf(0).data(0), 3)
        poke(uut.io.in.rf(1).data(0), 3)
        
        step(1)
        
        // Read
        poke(uut.io.control.rf(0).rAddrInt, 1)
        poke(uut.io.control.rf(1).rAddrInt, 1)
        expect(uut.io.out.rf(0).int, 1)
        expect(uut.io.out.rf(1).int, 1)
        
        poke(uut.io.control.rf(0).rAddrExt, 2)
        poke(uut.io.control.rf(1).rAddrExt, 2)
        expect(uut.io.out.rf(0).ext, 2)
        expect(uut.io.out.rf(1).ext, 2)
        
        poke(uut.io.control.rf(0).rAddrInt, 3)
        poke(uut.io.control.rf(1).rAddrInt, 3)
        expect(uut.io.out.rf(0).int, 3)
        expect(uut.io.out.rf(1).int, 3)
        
        // Bypass
        poke(uut.io.control.rf(0).bpSel.get, true)
        poke(uut.io.control.rf(1).bpSel.get, false)
        poke(uut.io.in.rf(0).data(0), 10)
        poke(uut.io.in.rf(1).data(0), 10)
        expect(uut.io.out.rf(0).int, 10)
        expect(uut.io.out.rf(1).int, 3)
        expect(uut.io.out.rf(0).ext, 10)
        expect(uut.io.out.rf(1).ext, 2)
    }
}

## Inner Product Unit

### Parallel Multiplier

#### Definition

In [ ]:
class PMultConfig(val numPairs: Int, val bitWidth: Int) {
    require(numPairs >= 1, "Must have at least one pair of multiplicands.")
    require(bitWidth >= 1, "Bitwidth must be at least one.")
}

class PMultInput(c: PMultConfig) extends Bundle {
    
    override def cloneType = (new PMultInput(c)).asInstanceOf[this.type]
    
    val weight = Vec(c.numPairs, SInt(c.bitWidth.W))
    val actvtn = Vec(c.numPairs, SInt(c.bitWidth.W))
}

class PMult(c: PMultConfig) extends Module {
    
    val io = IO(new Bundle {
        val in = Input(new PMultInput(c))
        val prod = Output(Vec(c.numPairs, SInt(c.bitWidth.W)))
    })
    
    io.prod := (io.in.weight zip io.in.actvtn).map { case(a, b) => a * b }
}

#### Verification

In [ ]:
val examplePMultCon = new PMultConfig(numPairs = 4, bitWidth = 8)

Driver(() => new PMult(examplePMultCon)) {
    uut => new PeekPokeTester(uut) {
        poke(uut.io.in.weight(0), 1) 
        poke(uut.io.in.actvtn(0), 2)
        
        poke(uut.io.in.weight(1), 3) 
        poke(uut.io.in.actvtn(1), 4)
        
        poke(uut.io.in.weight(2), 5)
        poke(uut.io.in.actvtn(2), 6)
        
        poke(uut.io.in.weight(3), 7)
        poke(uut.io.in.actvtn(3), 8)
        
        expect(uut.io.prod(0), 2)
        expect(uut.io.prod(1), 12)
        expect(uut.io.prod(2), 30)
        expect(uut.io.prod(3), 56)
  }
}

### Additive Reduction Tree

#### Definition

In [ ]:
class ARTreeConfig(val numAddends: Int, val bitWidth: Int) {
    require(numAddends >= 1, "Number of addends must be at least one.")
    require(bitWidth >= 1, "Bitwidth must be at least one.")
}

// Recursively creates a balanced syntax tree
def adjReduce[A](xs: List[A], op: (A, A) => A): A = xs match {
    case Nil => throw new IllegalArgumentException
    case List(single) => single
    case default => {
        val grouped = default.grouped(2).toList
        val result = for (g <- grouped) yield {
            g match {
                case List(a, b) => op(a, b)
                case List(x) => x
            }
        }
        adjReduce(result, op)
    }
}

class ARTree(c: ARTreeConfig) extends Module {
    
    val io = IO(new Bundle {
        val in  = Input(Vec(c.numAddends, SInt(c.bitWidth.W)))
        val sum = Output(SInt(c.bitWidth.W))
    })
    
    io.sum := adjReduce(io.in toList, (x: SInt, y: SInt) => x + y)
}

#### Verilog

In [ ]:
val exampleARTreeCon = new ARTreeConfig(4, 8)
println(getVerilog(new ARTree(exampleARTreeCon)))

#### Verification

In [ ]:
Driver(() => new ARTree(exampleARTreeCon)) {
    uut => new PeekPokeTester(uut) {
        poke(uut.io.in(0), 1) 
        poke(uut.io.in(1), 2)
        poke(uut.io.in(2), 8) 
        poke(uut.io.in(3), 9) 
        expect(uut.io.sum, 20)
        
        poke(uut.io.in(0), 1) 
        poke(uut.io.in(1), 2)
        poke(uut.io.in(2), 8) 
        poke(uut.io.in(3), 9) 
        expect(uut.io.sum, 20)
    }
}

### Putting them Together

#### Definition

In [ ]:
class IPUConfig(val width: Int, val bitWidth: Int, val bpType: String) {
    
    private val bypssError = "Bypass must be \"None\" or \"Firm\""
    private val widthError = "Width must be at least one"
    private val bitWdError = "Data bitwidth must be non-negative"
    
    val supportedBp = List("None", "Firm")
    
    require(width >= 1, widthError)
    require(supportedBp.contains(bpType), bypssError)
    require(bitWidth >= 0, bitWdError)
    
    val childPMultConfig = new PMultConfig(width, bitWidth)
    val childARTreeConfig = new ARTreeConfig(width, bitWidth)
    
    val bpFirm = (bpType == "Firm")
}

class IPUOutput(c: IPUConfig) extends Bundle {
    
    override def cloneType = (new IPUOutput(c)).asInstanceOf[this.type]
    
    // Disallowed. TODO: Read up on co/contravariance.
    // def this(ca: ALUConfig) = this(new IPUConfig(42, ca.dataWidth, ca.ipuBypassTp))
    
    val innerProd = Output(SInt(c.bitWidth.W))
    val bpWeight = if (c.bpFirm) Some(SInt(c.bitWidth.W)) else None
    val bpActvtn = if (c.bpFirm) Some(SInt(c.bitWidth.W)) else None
}


class IPU(c: IPUConfig) extends Module {
    
    val cPMConfig = c.childPMultConfig
    val cARTConfig = c.childARTreeConfig
    
    val io = IO(new Bundle {
        val dataIn = Input(new PMultInput(cPMConfig))
        val dataOut = Output(new IPUOutput(c))
        val bpSel = if (c.bpFirm) Some(Input(Vec(c.width, Bool()))) else None
    })
    
    val pMult = Module(new PMult(cPMConfig))
    pMult.io.in <> io.dataIn
    
    val aRTree = Module(new ARTree(cARTConfig))
    aRTree.io.in := pMult.io.prod
    
    io.dataOut.innerProd := aRTree.io.sum
    
    if (c.bpFirm) {
        io.dataOut.bpWeight.get := PriorityMux(io.bpSel.get, io.dataIn.weight)
        io.dataOut.bpActvtn.get := PriorityMux(io.bpSel.get, io.dataIn.actvtn)
    }
}

#### Verification

In [ ]:
val exampleIPUCon = new IPUConfig(width = 4, bitWidth = 8, bpType = "Firm")

Driver(() => new IPU(exampleIPUCon)) {
    uut => new PeekPokeTester(uut) {
        
        poke(uut.io.bpSel.get(0), 0)
        poke(uut.io.bpSel.get(1), 0)
        poke(uut.io.bpSel.get(2), 0)
        poke(uut.io.bpSel.get(3), 0)
        
        poke(uut.io.dataIn.weight(0), 1)
        poke(uut.io.dataIn.weight(1), 2)
        poke(uut.io.dataIn.weight(2), 3)
        poke(uut.io.dataIn.weight(3), 4)
        
        poke(uut.io.dataIn.actvtn(0), 5)
        poke(uut.io.dataIn.actvtn(1), 6)
        poke(uut.io.dataIn.actvtn(2), 7)
        poke(uut.io.dataIn.actvtn(3), 8)
        
        expect(uut.io.dataOut.innerProd, 70)
        
        poke(uut.io.bpSel.get(0), 0)
        poke(uut.io.bpSel.get(1), 1)
        poke(uut.io.bpSel.get(2), 0)
        poke(uut.io.bpSel.get(3), 0)
        
        expect(uut.io.dataOut.bpWeight.get, 2)
        expect(uut.io.dataOut.bpActvtn.get, 6)
        
        poke(uut.io.bpSel.get(0), 0)
        poke(uut.io.bpSel.get(1), 0)
        poke(uut.io.bpSel.get(2), 1)
        poke(uut.io.bpSel.get(3), 0)
        
        expect(uut.io.dataOut.bpWeight.get, 3)
        expect(uut.io.dataOut.bpActvtn.get, 7)
    }
}

## ALU

#### Definition

In [ ]:
class ALUConfig(val dataWidth: Int, val funcs: List[String]) {
    val identityError = "ALU functions must explicitly include Identity."
    val functionError = "Unsupported Error"
    val supportedFuncs = List("Identity", "Add", "Max", "Accumulate")
    
    require(funcs.contains("Identity"), identityError)
    for(x <- funcs) { require(supportedFuncs.contains(x), functionError) }
    
    val addSupp = funcs.contains("Add")
    val maxSupp = funcs.contains("Max")
    val accSupp = funcs.contains("Accumulate")
    val addBypassIn = addSupp || maxSupp
    val ipuBypassTp = if (addBypassIn) "Firm" else "None"
    val numFuncs = funcs.length
}

class ALUInput(c: ALUConfig) extends Bundle {
    
    override def cloneType = (new ALUInput(c)).asInstanceOf[this.type]
    
    val funcSel = Vec(c.numFuncs, Bool())
    val ipu = new IPUOutput(new IPUConfig(42, c.dataWidth, c.ipuBypassTp))
    val rfFeedback = if(c.accSupp) Some(SInt(c.dataWidth.W)) else None
}

class ALU(c: ALUConfig) extends Module {
 
    val io = IO(new Bundle {
        val in = Input(new ALUInput(c))
        val out = Output(SInt(c.dataWidth.W))
    })
    
    val idnOut = Some(Wire(SInt(c.dataWidth.W)))
    val addOut = if(c.addSupp) Some(Wire(SInt(c.dataWidth.W))) else None
    val maxOut = if(c.maxSupp) Some(Wire(SInt(c.dataWidth.W))) else None
    val accOut = if(c.accSupp) Some(Wire(SInt(c.dataWidth.W))) else None
    
    idnOut.get := io.in.ipu.innerProd
    
    if (c.addSupp) { addOut.get := io.in.ipu.bpWeight.get + io.in.ipu.bpActvtn.get }
    if (c.accSupp) { accOut.get := io.in.ipu.innerProd + io.in.rfFeedback.get }
    if (c.maxSupp) {
        when (io.in.ipu.bpWeight.get > io.in.ipu.bpActvtn.get) {
            maxOut.get := io.in.ipu.bpWeight.get
        } .otherwise {
            maxOut.get := io.in.ipu.bpActvtn.get
        }
    }
    
    val inters = (idnOut :: addOut :: maxOut :: accOut :: Nil) filter ( _.isDefined ) map ( _.get )
    io.out := PriorityMux(io.in.funcSel, inters)
}

#### Verification

In [ ]:
val exampleALUFuncs = "Identity" :: "Add" :: "Max" :: "Accumulate" :: Nil
val exampleALUCon = new ALUConfig(dataWidth = 8, funcs = exampleALUFuncs)

Driver(() => new ALU(exampleALUCon)) {
    uut => new PeekPokeTester(uut) {
        
        poke(uut.io.in.ipu.innerProd, 1)
        poke(uut.io.in.ipu.bpWeight.get, 2)
        poke(uut.io.in.ipu.bpActvtn.get, 3)
        poke(uut.io.in.rfFeedback.get, 4)
        
        poke(uut.io.in.funcSel(0), 1)
        poke(uut.io.in.funcSel(1), 0)
        poke(uut.io.in.funcSel(2), 0)
        poke(uut.io.in.funcSel(3), 0)
        expect(uut.io.out, 1)
        
        poke(uut.io.in.funcSel(0), 0)
        poke(uut.io.in.funcSel(1), 1)
        poke(uut.io.in.funcSel(2), 0)
        poke(uut.io.in.funcSel(3), 0)
        expect(uut.io.out, 5)
        
        poke(uut.io.in.funcSel(0), 0)
        poke(uut.io.in.funcSel(1), 0)
        poke(uut.io.in.funcSel(2), 1)
        poke(uut.io.in.funcSel(3), 0)
        expect(uut.io.out, 3)
        
        poke(uut.io.in.funcSel(0), 0)
        poke(uut.io.in.funcSel(1), 0)
        poke(uut.io.in.funcSel(2), 0)
        poke(uut.io.in.funcSel(3), 1)
        expect(uut.io.out, 5)
    }
}

## Nonlinear Unit

In [ ]:
class NLUConfig(val dataWidth: Int, val funcs: List[String]) {
    
    val supportedFuncs = List("Identity", "ReLu")
    val identityError = "NLU functions must explicitly include Identity."
    val functionError = "Unsupported Function"
    
    require(funcs.contains("Identity"), identityError)
    for(x <- funcs)(require(supportedFuncs.contains(x), functionError))
    
    val reluSupp = funcs.contains("ReLu")
    val numFuncs = funcs.length
}

class NLUInputs(c: NLUConfig) extends Bundle {
    
    override def cloneType = (new NLUInputs(c)).asInstanceOf[this.type]
    
    val data = SInt(c.dataWidth.W)
    val fSel = Vec(c.numFuncs, Bool())
}

class NLU(c: NLUConfig) extends Module {
    
    val io = IO(new Bundle {
        val in  = Input(new NLUInputs(c))
        val out = Output(SInt(c.dataWidth.W))
    })
    
    val idRes   = Some(Wire(SInt(c.dataWidth.W)))
    val reluRes = if(c.reluSupp) Some(Wire(SInt(c.dataWidth.W))) else None
    
    idRes.get := io.in.data
    
    if (c.reluSupp) {
        when (io.in.data > 0.S) {
            reluRes.get := io.in.data
        } .otherwise {
            reluRes.get := 0.S
        }
    }
    
    val inters = (idRes :: reluRes :: Nil) filter ( _.isDefined ) map ( _.get )
    io.out := PriorityMux(io.in.fSel, inters)
}

In [ ]:
val nluFuncs = "Identity" :: "ReLu" :: Nil
val nluCon = new NLUConfig(dataWidth = 8, funcs = nluFuncs)

Driver(() => new NLU(nluCon)) {
    uut => new PeekPokeTester(uut) {
        
        poke(uut.io.in.data, 5)
        
        poke(uut.io.in.fSel(0), 1)
        poke(uut.io.in.fSel(1), 0)
        expect(uut.io.out, 5)
        
        poke(uut.io.in.fSel(0), 0)
        poke(uut.io.in.fSel(1), 1)
        expect(uut.io.out, 5)
        
        poke(uut.io.in.data, -4)
        
        poke(uut.io.in.fSel(0), 1)
        poke(uut.io.in.fSel(1), 0)
        expect(uut.io.out, -4)
        
        poke(uut.io.in.fSel(0), 0)
        poke(uut.io.in.fSel(1), 1)
        expect(uut.io.out, 0)
    }
}

## Control

### State Machine

#### Definition

In [ ]:
class StateMachineConfig(
        val numStates: Int, 
        val numCtrlSigs: Int, 
        val stateMap: (UInt, UInt, StateMachineConfig) => UInt) {
    
    val stateWidth = log2Up(numStates)
    val ctrlWidth = log2Up(numCtrlSigs)
}

class StateMachine(c: StateMachineConfig) extends Module {
    
    val stateWidth: Int = log2Up(c.numStates)
    
    val io = IO(new Bundle {
        val control = Input (UInt(c.ctrlWidth.W))
        val out = Output(UInt(c.stateWidth.W))
    })
    
    val register = RegInit(0.U(c.stateWidth.W))
    register := c.stateMap(register, io.control, c)
    io.out := register
}

#### Example

In [ ]:
def exampleStateMap(state: UInt, control: UInt, c: StateMachineConfig): UInt = {
    
    val nextState = Wire(UInt(c.stateWidth.W))
    
    when      (state === 0.U & control === 0.U) { nextState := 0.U }
    .elsewhen (state === 0.U & control === 1.U) { nextState := 1.U }
    .elsewhen (state === 1.U & control === 0.U) { nextState := 0.U }
    .elsewhen (state === 1.U & control === 1.U) { nextState := 1.U }
    .otherwise { nextState := 0.U }
    
    nextState
}

#### Verification

In [ ]:
val exampleStateMachineConfig = new StateMachineConfig(2, 2, exampleStateMap)

Driver(() => new StateMachine(exampleStateMachineConfig)) {
    uut => new PeekPokeTester(uut) {
        poke(uut.io.control, 0)
        expect(uut.io.out, 0)
        
        // 0 -> 1
        poke(uut.io.control, 1)
        step(1)
        expect(uut.io.out, 1)
        
        // 1 -> 1
        poke(uut.io.control, 1)
        step(1)
        expect(uut.io.out, 1)
        
        // 1 -> 0
        poke(uut.io.control, 0)
        step(1)
        expect(uut.io.out, 0)
        
        // 0 -> 0
        poke(uut.io.control, 0)
        step(1)
        expect(uut.io.out, 0)
    }
}

### Decoder

#### Definition

In [ ]:
class PEConfig(
        val weightPRFConfig: PRFConfig,
        val actvtnPRFConfig: PRFConfig,
        val scratchRFConfig: RFConfig,
        val ipuConfig: IPUConfig,
        val aluConfig: ALUConfig,
        val nluConfig: NLUConfig,
        val smConfig: StateMachineConfig,
        val decodeWeightPRF: (UInt, PRFConfig) => Data,
        val decodeActvtnPRF: (UInt, PRFConfig) => Data,
        val decodeScratchRF: (UInt, RFConfig) => Data,
        val decodeIPU: (UInt, IPUConfig) => Data,
        val decodeALU: (UInt, ALUConfig) => Data,
        val decodeNLU: (UInt, NLUConfig) => Data)

class MemoryControl(c: PEConfig) extends Bundle {
    
    override def cloneType = (new MemoryControl(c)).asInstanceOf[this.type]
    
    val weightPRF = new PRFControl(c.weightPRFConfig)
    val actvtnPRF = new PRFControl(c.actvtnPRFConfig)
    val scratchRF = new RFControl(c.scratchRFConfig)
}

class ProcessControl(c: PEConfig) extends Bundle {
    
    override def cloneType = (new ProcessControl(c)).asInstanceOf[this.type]
    
    val aluFSel = Output(Vec(c.aluConfig.numFuncs, Bool()))
    val nluFSel = Output(Vec(c.nluConfig.numFuncs, Bool()))
    
    private val ipuPorts = c.weightPRFConfig.ports
    private val ipuBpFirm = c.ipuConfig.bpFirm
    val ipuBpSel = if (ipuBpFirm) Some(Output(Vec(ipuPorts, Bool()))) else None
}

class Decoder(c: PEConfig) extends Module {
    
    val io = IO(new Bundle {
        val state = Input(UInt(c.smConfig.stateWidth.W))
        val mem = Output(new MemoryControl(c))
        val proc = Output(new ProcessControl(c))
    })
    
    io.mem.weightPRF <> c.decodeWeightPRF(io.state, c.weightPRFConfig)
    io.mem.actvtnPRF <> c.decodeActvtnPRF(io.state, c.actvtnPRFConfig)
    io.mem.scratchRF <> c.decodeScratchRF(io.state, c.scratchRFConfig)
    
    if (c.ipuConfig.bpFirm) { 
        io.proc.ipuBpSel.get := c.decodeIPU(io.state, c.ipuConfig)
    }
    
    io.proc.aluFSel := c.decodeALU(io.state, c.aluConfig)
    io.proc.nluFSel := c.decodeNLU(io.state, c.nluConfig)
}

#### Example

In [ ]:
def exampleDecodeWeightPRF(state: UInt, c: PRFConfig) = {
    
    val data = Wire(new PRFControl(c))
    
    when (state === 0.U) {
        data.rf.foreach { k =>
            k.wEnable   := true.B
            k.rEnable   := true.B
            k.wAddr     := 1.U
            k.rAddrInt  := 2.U
            k.rAddrExt  := 3.U
            if (k.bpSel.isDefined) { k.bpSel.get := true.B }
        }
    } .otherwise {
        data.rf.foreach { k =>
            k.wEnable   := false.B
            k.rEnable   := false.B
            k.wAddr     := 4.U
            k.rAddrInt  := 5.U
            k.rAddrExt  := 6.U
            if (k.bpSel.isDefined) { k.bpSel.get := false.B }
        }
    }
    
    data
}

def exampleDecodeActvtnPRF(state: UInt, c: PRFConfig) = {
    
    val data = Wire(new PRFControl(c))
    
    when (state === 0.U) {
        data.rf.foreach { k =>
            k.wEnable   := true.B
            k.rEnable   := true.B
            k.wAddr     := 1.U
            k.rAddrInt  := 2.U
            k.rAddrExt  := 3.U
            if (k.bpSel.isDefined) { k.bpSel.get := true.B }
        }
    } .otherwise {
        data.rf.foreach { k =>
            k.wEnable   := false.B
            k.rEnable   := false.B
            k.wAddr     := 4.U
            k.rAddrInt  := 5.U
            k.rAddrExt  := 6.U
            if (k.bpSel.isDefined) { k.bpSel.get := false.B }
        }
    }
    
    data
}

def exampleDecodeScratchRF(state: UInt, c: RFConfig) = {
    
    val data = Wire(new RFControl(c))
    
    when (state === 0.U) {
        data.wEnable   := true.B
        data.rEnable   := true.B
        data.wAddr     := 1.U
        data.rAddrInt  := 2.U
        data.rAddrExt  := 3.U
        if(data.bpSel.isDefined) { data.bpSel.get := true.B }
        if(data.inSel.isDefined) { data.inSel.get := Vec(List(true.B, false.B)) }
    } .otherwise {
        data.wEnable   := false.B
        data.rEnable   := false.B
        data.wAddr     := 4.U
        data.rAddrInt  := 5.U
        data.rAddrExt  := 6.U
        if(data.bpSel.isDefined) { data.bpSel.get := false.B }
        if(data.inSel.isDefined) { data.inSel.get := Vec(List(false.B, true.B)) }
    }
    
    data
}

def exampleDecodeIPU(state: UInt, c: IPUConfig) = {
    
    val data = Wire(Vec(c.width, Bool()))
    
    when (state === 0.U) {
        data := Vec(1.U :: 0.U :: Nil)
    } .otherwise {
        data := Vec(0.U :: 1.U :: Nil)
    }
    
    data
}

def exampleDecodeALU(state: UInt, c: ALUConfig) = {
    
    val data = Wire(Vec(c.numFuncs, Bool()))
    
    when (state === 0.U) {
        data := Vec(1.U :: 0.U :: 0.U :: 0.U :: Nil)
    } .otherwise {
        data := Vec(0.U :: 1.U :: 0.U :: 0.U :: Nil)
    }
    
    data
}

def exampleDecodeNLU(state: UInt, c: NLUConfig) = {
    
    val data = Wire(Vec(c.numFuncs, Bool()))
    
    when (state === 0.U) {
        data := Vec(1.U :: 0.U :: Nil)
    } .otherwise {
        data := Vec(0.U :: 1.U :: Nil)
    }
    
    data
}


#### Verification

In [ ]:
// TODO: require IPU width == weightPRF width == actvtnPRF width
// TODO: require IPUConfig "Firm" if ALUConfig "Add" or "Max"

val examplePEConfig = new PEConfig(
    new PRFConfig(2, 8, 4, 1, "Soft"),
    new PRFConfig(2, 8, 4, 1, "Soft"),
    new RFConfig(8, 4, 2, true),
    new IPUConfig(2, 8, "Firm"),
    new ALUConfig(8, List("Identity", "Add", "Max", "Accumulate")),
    new NLUConfig(8, List("Identity", "ReLu")),
    new StateMachineConfig(4, 4, exampleStateMap),
    exampleDecodeWeightPRF,
    exampleDecodeActvtnPRF,
    exampleDecodeScratchRF,
    exampleDecodeIPU,
    exampleDecodeALU,
    exampleDecodeNLU
)


Driver(() => new Decoder(examplePEConfig)) {
    
    uut => new PeekPokeTester(uut) {
        
        poke(uut.io.state, 0.U)
        step(1)
        
        expect(uut.io.mem.weightPRF.rf(0).wEnable, true.B)
        expect(uut.io.mem.weightPRF.rf(0).rEnable, true.B)
        expect(uut.io.mem.weightPRF.rf(0).wAddr, 1.U)
        expect(uut.io.mem.weightPRF.rf(0).rAddrInt, 2.U)
        expect(uut.io.mem.weightPRF.rf(0).rAddrExt, 3.U)
        expect(uut.io.mem.weightPRF.rf(0).bpSel.get, true.B)
        
        expect(uut.io.mem.actvtnPRF.rf(0).wEnable, true.B)
        expect(uut.io.mem.actvtnPRF.rf(0).rEnable, true.B)
        expect(uut.io.mem.actvtnPRF.rf(0).wAddr, 1.U)
        expect(uut.io.mem.actvtnPRF.rf(0).rAddrInt, 2.U)
        expect(uut.io.mem.actvtnPRF.rf(0).rAddrExt, 3.U)
        expect(uut.io.mem.actvtnPRF.rf(0).bpSel.get, true.B)
        
        expect(uut.io.proc.ipuBpSel.get(0), 1)
        expect(uut.io.proc.ipuBpSel.get(1), 0)
        
        expect(uut.io.proc.aluFSel(0), 1)
        expect(uut.io.proc.aluFSel(1), 0)
        expect(uut.io.proc.aluFSel(2), 0)
        expect(uut.io.proc.aluFSel(3), 0)
        
        expect(uut.io.mem.sratchRF.rf(0).wEnable, true.B)
        expect(uut.io.mem.sratchRF.rf(0).rEnable, true.B)
        expect(uut.io.mem.sratchRF.rf(0).wAddr, 1.U)
        expect(uut.io.mem.sratchRF.rf(0).rAddrInt, 2.U)
        expect(uut.io.mem.sratchRF.rf(0).rAddrExt, 3.U)
        expect(uut.io.mem.sratchRF.rf(0).bpSel.get, true.B)
        
        expect(uut.io.proc.nluFSel(0), 1)
        expect(uut.io.proc.nluFSel(1), 0)
        
        poke(uut.io.state, 1.U) 
        step(1)
        
        expect(uut.io.mem.weightPRF.rf(0).wEnable, false.B)
        expect(uut.io.mem.weightPRF.rf(0).rEnable, false.B)
        expect(uut.io.mem.weightPRF.rf(0).wAddr, 4.U)
        expect(uut.io.mem.weightPRF.rf(0).rAddrInt, 5.U)
        expect(uut.io.mem.weightPRF.rf(0).rAddrExt, 6.U)
        expect(uut.io.mem.weightPRF.rf(0).bpSel.get, false.B)
        
        expect(uut.io.mem.actvtnPRF.rf(0).wEnable, false.B)
        expect(uut.io.mem.actvtnPRF.rf(0).rEnable, false.B)
        expect(uut.io.mem.actvtnPRF.rf(0).wAddr, 4.U)
        expect(uut.io.mem.actvtnPRF.rf(0).rAddrInt, 5.U)
        expect(uut.io.mem.actvtnPRF.rf(0).rAddrExt, 6.U)
        expect(uut.io.mem.actvtnPRF.rf(0).bpSel.get, false.B)
        
        expect(uut.io.proc.ipuBpSel.get(0), 0)
        expect(uut.io.proc.ipuBpSel.get(1), 1)
        
        expect(uut.io.proc.aluFSel(0), 0)
        expect(uut.io.proc.aluFSel(1), 1)
        expect(uut.io.proc.aluFSel(2), 0)
        expect(uut.io.proc.aluFSel(3), 0)
        
        expect(uut.io.mem.scratchRF.rf(0).wEnable, false.B)
        expect(uut.io.mem.scratchRF.rf(0).rEnable, false.B)
        expect(uut.io.mem.scratchRF.rf(0).wAddr, 4.U)
        expect(uut.io.mem.scratchRF.rf(0).rAddrInt, 5.U)
        expect(uut.io.mem.scratchRF.rf(0).rAddrExt, 6.U)
        expect(uut.io.mem.scratchRF.rf(0).bpSel.get, false.B)
        
        expect(uut.io.proc.nluFSel(0), 0)
        expect(uut.io.proc.nluFSel(1), 1)
        
    }
}


## PE

#### Definition

In [ ]:
class PE(c: PEConfig) extends Module {
    
    val cw = c.weightPRFConfig
    val ca = c.actvtnPRFConfig
    val cs = c.scratchRFConfig
    
    val io = IO(new Bundle {
        val stateCtrl = Input(UInt(c.smConfig.ctrlWidth.W))
        val toWeightPRF = Input(new PRFInput(cw))
        val toActvtnPRF = Input(new PRFInput(ca))
        val toScratchRF = Input(SInt(cs.dataWidth.W))
        val fromWeightPRF = Output(Vec(cw.ports, SInt(cw.dataWidth.W)))
        val fromActvtnPRF = Output(Vec(ca.ports, SInt(ca.dataWidth.W)))
        val fromScratchRF = Output(SInt(cs.dataWidth.W))
        val totalOutput = Output(SInt(c.nluConfig.dataWidth.W))
    })
    
    val stateMachine = Module(new StateMachine(c.smConfig))
    stateMachine.io.control := io.stateCtrl
    
    val decoder = Module(new Decoder(c))
    decoder.io.state := stateMachine.io.out
    
    val weightPRF = Module(new PRF(cw))
    weightPRF.io.control <> decoder.io.mem.weightPRF
    weightPRF.io.in <> io.toWeightPRF
    weightPRF.io.out.rf.zipWithIndex.map { 
        case (x: RFOutput, i: Int) => io.fromWeightPRF(i) := x.ext
    }
    
    val actvtnPRF = Module(new PRF(ca))
    actvtnPRF.io.control <> decoder.io.mem.actvtnPRF
    actvtnPRF.io.in <> io.toActvtnPRF
    actvtnPRF.io.out.rf.zipWithIndex.map {
        case (x: RFOutput, i: Int) => io.fromActvtnPRF(i) := x.ext
    }
       
    val ipu = Module(new IPU(c.ipuConfig))
    if (ipu.io.bpSel.isDefined) { ipu.io.bpSel.get := decoder.io.proc.ipuBpSel.get }
    weightPRF.io.out.rf.zipWithIndex.map { 
        case (x: RFOutput, i: Int) => ipu.io.dataIn.weight(i) := x.int
    }
    actvtnPRF.io.out.rf.zipWithIndex.map {
        case (x: RFOutput, i: Int) => ipu.io.dataIn.actvtn(i) := x.int
    }

    val alu = Module(new ALU(c.aluConfig))
    alu.io.in.funcSel := decoder.io.proc.aluFSel
    alu.io.in.ipu <> ipu.io.dataOut
    
    val scratchRF = Module(new RF(cs))
    scratchRF.io.control <> decoder.io.mem.scratchRF
    // This next group of statements is the result of poor decisions :(
    scratchRF.io.in.data(0) := io.toScratchRF
    scratchRF.io.in.data(1) := alu.io.out
    io.fromScratchRF := scratchRF.io.out.ext
    if(alu.io.in.rfFeedback.isDefined) alu.io.in.rfFeedback.get := scratchRF.io.out.int
    
    val nlu = Module(new NLU(c.nluConfig))
    nlu.io.in.fSel := decoder.io.proc.nluFSel
    nlu.io.in.data := scratchRF.io.out.int
    io.totalOutput := nlu.io.out
}

#### Verification

In [ ]:
val examplePEConfig10 = new PEConfig(
    new PRFConfig(2, 8, 4, 1, "Soft"),
    new PRFConfig(2, 8, 4, 1, "Soft"),
    new RFConfig(8, 4, 2, false),
    new IPUConfig(2, 8, "Firm"),
    new ALUConfig(8, List("Identity", "Add", "Max", "Accumulate")),
    new NLUConfig(8, List("Identity", "ReLu")),
    new StateMachineConfig(4, 4, exampleStateMap),
    exampleDecodeWeightPRF,
    exampleDecodeActvtnPRF,
    exampleDecodeScratchRF,
    exampleDecodeIPU,
    exampleDecodeALU,
    exampleDecodeNLU
)

Driver(() => new PE(examplePEConfig10)) {
    uut => new PeekPokeTester(uut) {
        
    }
}

## Future Plans
* Verify everything using Golden Models